# Uncertainty Analysis with NZUpy's Range Runs

## Overview of Range Runs

NZUpy allows for uncertainty analysis through "range runs," which enable the evaluation of sensitivities in gross emissions response to carbon pricing. This notebook demonstrates:

1. How to set up and run uncertainty analyses
2. Creating a policy scenario with custom industrial allocation phase-out
3. Generating and interpreting range charts that show uncertainty bands
4. Using interactive visualisation capabilities of NZUpy

Let's begin by importing the necessary libraries and setting up our model.

## 1. Set Up the NZUpy Model

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

# Add the project root to the path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import the NZUpy class
from model.core.base_model import NZUpy
from model.utils.chart_generator import ChartGenerator
from model.interface.chart_display import create_chart_page

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "03_range_runs"
os.makedirs(output_dir, exist_ok=True)

In [2]:
# Initialise NZUpy
NZU = NZUpy(data_dir=data_dir)

## 2. Define Time Periods and Set Up Range Scenario Type

For uncertainty analysis, we need to set the model to use "Range" scenario type, which automatically configures the model to run with different demand sensitivities.


In [3]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

# Set the scenario type to 'Range' for uncertainty analysis
NZU.define_scenario_type('Range')

# Prime the model - this will automatically set up the 5 sensitivity levels
NZU.prime()

Time periods defined:
  Optimisation: 2024-2050
Scenario type set to 'Range'
Model primed with 5 scenarios:
  [0] 95% Lower
  [1] 1 s.e lower
  [2] central
  [3] 1 s.e upper
  [4] 95% Upper


## 3. Configure Policy Scenario

We'll set up an interesting policy scenario with two key modifications:
1. Low forestry configuration - representing reduced forestry planting
2. Custom industrial allocation phase-out by 2040 - representing industrial allocation reform

In [4]:
# First, let's use the central configuration for all components as our starting point
NZU.configure_range_scenarios()

# Then apply the low forestry config
NZU.use_config(0, 'forestry', 'low')
print("Applied 'low' forestry configuration")


Set emissions configuration to 'central' for scenario '95% Lower'
Set auction configuration to 'central' for scenario '95% Lower'
Set industrial allocation configuration to 'central' for scenario '95% Lower'
Set forestry configuration to 'central' for scenario '95% Lower'
Set demand model number to 2 for scenario '95% Lower'
Set demand sensitivity to 'central' for scenario '95% Lower'
Using central config for stockpile in scenario 0 (95% Lower)
Using central configs for all components in model scenario 0 (95% Lower)
Set demand model number to 2 for scenario '95% Lower'
Set demand sensitivity to '95% Lower' for scenario '95% Lower'
Set emissions configuration to 'central' for scenario '1 s.e lower'
Set auction configuration to 'central' for scenario '1 s.e lower'
Set industrial allocation configuration to 'central' for scenario '1 s.e lower'
Set forestry configuration to 'central' for scenario '1 s.e lower'
Set demand model number to 2 for scenario '1 s.e lower'
Set demand sensitivity t

In [5]:
# Now let's create a custom industrial allocation phase-out schedule
print("\nSetting up industrial allocation phase-out...")

# Get the current industrial allocation data
industrial_baseline_data = NZU.data_handler.get_industrial_allocation_data(config='central')

# Check what we got
print("\nRaw Industrial Allocation Data Structure:")
print(industrial_baseline_data.head())

# Create our phase-out schedule from 2024 (100%) to 2040 (0%)
phase_out_years = range(2024, 2041)
phase_out_factors = np.linspace(1.0, 0.0, len(phase_out_years))

# Create a new DataFrame for our phased allocation
phased_allocation = industrial_baseline_data.copy()

# Apply the phase-out factors to the baseline allocation
for i, year in enumerate(phase_out_years):
    if year in phased_allocation.index:
        phased_allocation.loc[year, 'baseline_allocation'] = industrial_baseline_data.loc[year, 'baseline_allocation'] * phase_out_factors[i]

# Set any years after 2040 to zero
for year in phased_allocation.index:
    if year > 2040:
        phased_allocation.loc[year, 'baseline_allocation'] = 0.0

# Update the industrial allocation series in the model
NZU.set_series('baseline_allocation', phased_allocation['baseline_allocation'], 'industrial')


Setting up industrial allocation phase-out...

Raw Industrial Allocation Data Structure:
      baseline_allocation  activity_adjustment
year                                          
2020          7638.401600                  1.0
2021          6404.526600                  1.0
2022          6146.079000                  1.0
2023          6067.094999                  1.0
2024          6058.110998                  1.0
Updated industrial.baseline_allocation for scenario '95% Lower'


In [6]:
# Print the modified industrial allocation to verify
print("\nModified industrial allocation with phase-out by 2040:")
print(phased_allocation.loc[[2024, 2030, 2035, 2040, 2045]])


Modified industrial allocation with phase-out by 2040:
      baseline_allocation  activity_adjustment
year                                          
2024          6058.110998                  1.0
2030          2928.459897                  1.0
2035          1276.728037                  1.0
2040             0.000000                  1.0
2045             0.000000                  1.0


## 4. Run the Model with Uncertainty Analysis

Now let's run the model with our configured range scenario.

In [7]:
# Run the model
results = NZU.run()

Set emissions configuration to 'central' for scenario '95% Lower'
Set auction configuration to 'central' for scenario '95% Lower'
Set industrial allocation configuration to 'central' for scenario '95% Lower'
Set forestry configuration to 'central' for scenario '95% Lower'
Set demand model number to 2 for scenario '95% Lower'
Set demand sensitivity to 'central' for scenario '95% Lower'
Using central config for stockpile in scenario 0 (95% Lower)
Using central configs for all components in model scenario 0 (95% Lower)
Set demand model number to 2 for scenario '95% Lower'
Set demand sensitivity to '95% Lower' for scenario '95% Lower'
Set emissions configuration to 'central' for scenario '1 s.e lower'
Set auction configuration to 'central' for scenario '1 s.e lower'
Set industrial allocation configuration to 'central' for scenario '1 s.e lower'
Set forestry configuration to 'central' for scenario '1 s.e lower'
Set demand model number to 2 for scenario '1 s.e lower'
Set demand sensitivity t

## 5. Generate Range Charts

One of the key advantages of range runs is the ability to generate charts that show uncertainty bands around central estimates. Let's explore this capability.

In [ ]:
# Initialise chart generator
chart_gen = ChartGenerator(NZU)

# Generate carbon price chart with uncertainty bands
print("Generating carbon price chart with uncertainty bands...")
price_chart = chart_gen.carbon_price_chart()
price_chart.update_layout(title="Carbon Price Projection with Uncertainty Bands")
price_chart.show()
price_chart.write_image(str(output_dir / "range_price_chart.png"))

# Generate emissions pathway chart with uncertainty bands
print("\nGenerating emissions pathway chart with uncertainty bands...")
emissions_chart = chart_gen.emissions_pathway_chart()
emissions_chart.update_layout(title="Emissions Pathway with Uncertainty Bands")
emissions_chart.show()
emissions_chart.write_image(str(output_dir / "range_emissions_chart.png"))

# Generate stockpile balance chart with uncertainty bands
print("\nGenerating stockpile balance chart with uncertainty bands...")
stockpile_chart = chart_gen.stockpile_balance_chart()
stockpile_chart.update_layout(title="Stockpile Balance with Uncertainty Bands")
stockpile_chart.show()
stockpile_chart.write_image(str(output_dir / "range_stockpile_chart.png"))

## 6. Create Interactive HTML Dashboard

NZUpy provides functionality to create interactive HTML dashboards with multiple charts. Let's create one for our range run.

In [ ]:
# Create a collection of charts for our dashboard
chart_collection = {
    "Carbon Price Projection": price_chart,
    "Emissions Pathway": emissions_chart,
    "Stockpile Balance": stockpile_chart
}

# Create an interactive HTML dashboard
dashboard_html = create_chart_page(
    charts=chart_collection,
    title="NZUpy Range Run Analysis",
    subtitle="Uncertainty Analysis with Industrial Phase-out and Low Forestry",
    output_dir=output_dir,
    filename="range_run_dashboard.html"
)

print(f"\nInteractive dashboard saved to: {output_dir / 'range_run_dashboard.html'}")
print("Open this file in a web browser to explore the interactive charts")

## 7. Extract and Analyse Uncertainty Ranges

Let's extract the data and analyse the uncertainty ranges for key metrics.

In [ ]:
# Extract carbon price data for all sensitivity levels
print("\nExtracted carbon price data for uncertainty analysis:")
price_data = NZU.prices.xs('carbon_price', level='variable', axis=1)
print(price_data.head())

# Calculate the width of uncertainty bands at different years
forecast_years = [2030, 2035, 2040, 2045, 2050]

print("\nUncertainty ranges for carbon prices ($/tCO2e):")
print(f"{'Year':<10} {'Central':<10} {'±1 Std Err':<15} {'95% Range':<20}")
print("-" * 55)

for year in forecast_years:
    if year in price_data.index:
        central = price_data['central'][year]
        lower_std = price_data['1 s.e lower'][year]
        upper_std = price_data['1 s.e upper'][year]
        lower_95 = price_data['95% Lower'][year]
        upper_95 = price_data['95% Upper'][year]
        
        std_range = f"[{lower_std:.2f}, {upper_std:.2f}]"
        conf_range = f"[{lower_95:.2f}, {upper_95:.2f}]"
        
        print(f"{year:<10} {central:<10.2f} {std_range:<15} {conf_range:<20}")

# Now do the same for emissions
emissions_data = NZU.demand.xs('emissions', level='variable', axis=1)

print("\nUncertainty ranges for emissions (kt CO2e):")
print(f"{'Year':<10} {'Central':<10} {'±1 Std Err':<15} {'95% Range':<20}")
print("-" * 55)

for year in forecast_years:
    if year in emissions_data.index:
        central = emissions_data['central'][year]
        lower_std = emissions_data['1 s.e lower'][year]
        upper_std = emissions_data['1 s.e upper'][year]
        lower_95 = emissions_data['95% Lower'][year]
        upper_95 = emissions_data['95% Upper'][year]
        
        std_range = f"[{lower_std:.0f}, {upper_std:.0f}]"
        conf_range = f"[{lower_95:.0f}, {upper_95:.0f}]"
        
        print(f"{year:<10} {central:<10.0f} {std_range:<15} {conf_range:<20}")


## 8. Create a Custom Chart Comparing Baseline and Price-Responsive Emissions

Let's create a custom chart comparing baseline emissions (before price response) with the central estimate of price-responsive emissions.

In [ ]:
# Extract baseline emissions and price-responsive emissions
baseline_emissions = NZU.demand.xs('baseline', level='variable', axis=1)['central']
responsive_emissions = NZU.demand.xs('emissions', level='variable', axis=1)['central']

# Create a custom figure
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(
    x=baseline_emissions.index,
    y=baseline_emissions.values,
    name="Baseline Emissions (Before Price Response)",
    line=dict(color="rgb(204, 85, 68)", width=3, dash='dash')
))

fig.add_trace(go.Scatter(
    x=responsive_emissions.index,
    y=responsive_emissions.values,
    name="Emissions After Price Response (Central Estimate)",
    line=dict(color="rgb(68, 114, 196)", width=3)
))

# Calculate and add gross mitigation as a filled area
fig.add_trace(go.Scatter(
    x=baseline_emissions.index.tolist() + baseline_emissions.index.tolist()[::-1],
    y=baseline_emissions.values.tolist() + responsive_emissions.values.tolist()[::-1],
    fill='toself',
    fillcolor='rgba(68, 114, 196, 0.2)',
    line=dict(width=0),
    name="Gross Mitigation",
    showlegend=True
))

# Update layout
fig.update_layout(
    title="Impact of Carbon Price on Emissions",
    xaxis_title="Year",
    yaxis_title="Emissions (kt CO₂-e)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Show and save the figure
fig.show()
fig.write_image(str(output_dir / "custom_emissions_mitigation_chart.png"))

## 9. Discussion of Range Run Results

The range charts provide valuable insights about both the expected outcomes and the uncertainty around our policy scenario:

1. **Carbon Price Uncertainty**: The carbon price chart shows the central estimate along with two uncertainty bands: the darker band represents one standard error around the central estimate, while the lighter band represents the 95% confidence interval. This demonstrates the substantial uncertainty in price projections, particularly in later years.

2. **Policy Effects**: Our policy scenario, with its combination of lower forestry removals and phased-out industrial allocation, leads to higher carbon prices due to reduced unit supply. This illustrates how policy choices can impact market outcomes.

3. **Emissions Pathways**: The emissions pathway chart shows how different demand sensitivities result in varying levels of emissions reduction in response to price signals. The uncertainty bands widen over time, reflecting increased uncertainty about price elasticity of emissions in the longer term.

4. **Stockpile Dynamics**: The stockpile balance chart illustrates how the stockpile is expected to evolve under different demand sensitivities. The uncertainty in demand response leads to different patterns of stockpile usage.

5. **Gross Mitigation**: Our custom chart shows the extent of gross mitigation (difference between baseline and price-responsive emissions) that occurs due to carbon pricing. This is a key metric for evaluating policy effectiveness.

The uncertainty bands provide crucial context for decision-making, showing the range of potential outcomes rather than just point estimates. This acknowledges the inherent uncertainties in emissions response to price signals and helps policymakers understand the robustness of projected outcomes.


## 10. Conclusion

This notebook has demonstrated NZUpy's capability to perform uncertainty analysis through range runs. We've seen how to:

1. **Set up a range scenario** for uncertainty analysis
2. **Configure a custom policy setting** with industrial allocation phase-out and low forestry
3. **Generate and interpret range charts** with uncertainty bands
4. **Create interactive dashboards** for exploring results
5. **Extract and analyse uncertainty ranges** for key metrics
6. **Build custom visualisations** to highlight specific aspects of the results

Range runs are a powerful tool for robust policy analysis, allowing stakeholders to understand not just the expected outcomes of different settings but also the uncertainty around those projections. This approach provides a more complete picture for decision-making in the complex and evolving landscape of emissions trading.

The interactive nature of NZUpy's visualisations enhances the ability to explore and understand these complex results, making it easier to communicate findings to stakeholders.